In [123]:
import matplotlib.pyplot as plt
import vtk
import numpy as np
import sys
import math
import os
import glob
from vtk.util.numpy_support import *
import pandas
from multiprocessing import Pool
from vtk.util import numpy_support
from sklearn.decomposition import PCA
from numpy import linalg as LA
from skimage.measure import EllipseModel
from matplotlib.patches import Ellipse
import pickle

In [124]:
def ensureUtf(s):
    try:
        if type(s) == unicode:
            return s.encode('utf8', 'ignore')
    except: 
        return str(s)

def read_vti(filename):
    reader = vtk.vtkXMLImageDataReader()
    reader.SetFileName(filename)
    reader.Update()
    return reader.GetOutput()

def write_vti(filename,data):
    writer = vtk.vtkXMLImageDataWriter()
    writer.SetInputData(data)
    writer.SetFileName(filename)
    writer.Update()
    
def write_vtu(filename,data):
    writer = vtk.vtkXMLUnstructuredGridWriter()
    writer.SetInputData(data)
    writer.SetFileName(filename)
    writer.Update()    
    
def write_multiblock(filename,data):
    writer = vtk.vtkXMLMultiBlockDataWriter()
    writer.SetInputData(data)
    writer.SetFileName(filename)
    writer.Update()    
    
def compute_bubble_features(fname,confidence_th,size_threshold, tstep, outpath):
    
    feature_values=[]
    
    if tstep%10 == 0:
        print ('processing time step: ' + str(tstep))
        
    data = read_vti(fname)
    data.GetPointData().SetActiveScalars('feature_similarity')
    
    gbounds = data.GetBounds()
    #print ('Bounds: ' + str(gbounds))
    
    thresholding = vtk.vtkThreshold()
    thresholding.ThresholdByUpper( confidence_th )
    thresholding.SetInputData(data)

    seg = vtk.vtkConnectivityFilter()
    seg.SetInputConnection(thresholding.GetOutputPort())
    seg.SetExtractionModeToLargestRegion()
    seg.Update()

    segmentation = vtk.vtkConnectivityFilter()
    segmentation.SetInputConnection(thresholding.GetOutputPort())
    segmentation.SetExtractionModeToAllRegions()
    segmentation.ColorRegionsOn()
    segmentation.Update()

    ug = segmentation.GetOutput()
    num_segments = segmentation.GetNumberOfExtractedRegions()
    
    ## compute volumes of each bubble: 
    ## volume = voxel count
    bubble_volumes = np.zeros(num_segments)
    for i in range(ug.GetPointData().GetArray('RegionId').GetNumberOfTuples()):
        regionId = int(ug.GetPointData().GetArray('RegionId').GetTuple(i)[0])
        bubble_volumes[regionId] = bubble_volumes[regionId]+1
        
    num_valid_features=0
    for i in range(num_segments):
        if  bubble_volumes[i] > size_threshold:
            num_valid_features=num_valid_features+1
            
    #print ("Number of valid features: " + str(num_valid_features))        
    
    find_max_topmost_xvals=[]
    idx=0
    for i in range(num_segments):
        if  bubble_volumes[i] > size_threshold:
            #print ('processing bubble: '+ str(i))
            thresholding2 = vtk.vtkThreshold()
            thresholding2.SetInputData(ug)
            thresholding2.ThresholdBetween(i,i)
            thresholding2.Update()

            single_feature = thresholding2.GetOutput()
            
            bounds = single_feature.GetBounds()
            aspect_ratio_3d = (bounds[5] - bounds[4])/(bounds[1] - bounds[0])
            #print ('aspect ratio: ' + str(aspect_ratio_3d))
            
            feature_pts = numpy_support.vtk_to_numpy(single_feature.GetPoints().GetData())
            
            ## compute centroid
            x_center = np.mean(feature_pts[:,0])
            y_center = np.mean(feature_pts[:,1])
            z_center = np.mean(feature_pts[:,2])
            
            max_x = np.max(feature_pts[:,0])
            find_max_topmost_xvals.append([max_x,i,idx])
            
            ## only x and z components are extracted
            feature_pts_2d = feature_pts[:, [0, 2]]
            ## Do PCA to project to 2D
            pca = PCA(n_components=2)
            principalComponents = pca.fit_transform(feature_pts_2d)
            eigenvalues = pca.explained_variance_
            roundness = np.sqrt(1-((eigenvalues[1]*eigenvalues[1])/(eigenvalues[0]*eigenvalues[0])))
            #print ('eigen ratio: ' + str(roundness))
            
            #print ('volume: ' + str(bubble_volumes[i]))
            #print()
            
            feature_values.append([tstep,i,idx,aspect_ratio_3d,roundness,bubble_volumes[i],x_center,y_center,z_center])
            
            idx=idx+1
            
            
    ## identify the indices of features which are actually topmost and not desired bubbles
    find_max_topmost_xvals = np.asarray(find_max_topmost_xvals)
    top_indices=[]
    for i in range(len(find_max_topmost_xvals)):
        diff = np.abs(gbounds[1]-find_max_topmost_xvals[i,0])
        if diff < 0.00005:
            top_indices.append(int(find_max_topmost_xvals[i,1]))
    
    #print ('feature_values len: ' + str(len(feature_values)))
    #print (feature_values)
    #print ('top indices: ' + str(top_indices))
    
    ## filter out the top most undesired features and write the file out
    mb = vtk.vtkMultiBlockDataSet()
    mb.SetNumberOfBlocks(num_valid_features)
    idx=0
    cnt=0
    final_filtered_feature_values=[]
    for i in range(num_segments):
        
        if  bubble_volumes[i] > size_threshold and i not in(top_indices):
            thresholding2 = vtk.vtkThreshold()
            thresholding2.SetInputData(ug)
            thresholding2.ThresholdBetween(i,i)
            thresholding2.Update()
            single_feature = thresholding2.GetOutput()
            mb.SetBlock(idx,single_feature)
            
            final_filtered_feature_values.append(feature_values[cnt])

            idx=idx+1
            
        if  bubble_volumes[i] > size_threshold:
            cnt=cnt+1
            
    file_name = outpath + 'segmented_' + str(tstep) + '.vtm'
    write_multiblock(file_name, mb)
        
    #print ('final_filtered_feature_values len: ' + str(len(final_filtered_feature_values)))
    #print (final_filtered_feature_values)
    
    return final_filtered_feature_values

In [125]:
## Parameters
init_time = 75
end_time = 78
input_path = '../out/mfix_local_grid/'
outpath = '../out/mfix_local_grid/segmented_features_1/'
confidence_th = 0.9
size_threshold = 100

In [126]:
## Run for all timesteps
for ii in range(init_time,end_time):
    inpfname = input_path + 'slic_compare_' + str(ii) + '.vti'
    feature_values_ret = compute_bubble_features(inpfname,confidence_th,size_threshold, ii, outpath)
    
    outpicklefname = outpath + 'feature_values_' + str(ii) + '.pickle'
    pickle.dump(feature_values_ret, open( outpicklefname, "wb"))

In [127]:
### stack all data to generate csv file for PCP
outfname = outpath + 'feature_values.csv'
#outfname = '/Users/sdutta/Desktop/feature_values.csv'
all_features = np.array([0,0,0,0,0,0,0,0,0])

## Reload feature values from pickle file
for ii in range(init_time,end_time):    
    inpicklefname = outpath + 'feature_values_' + str(ii) + '.pickle'
    #print (inpicklefname)
    ffname = ensureUtf(inpicklefname)
    fname = open(ffname, 'rb')
    data = pickle.load(fname, encoding='latin1')
    data = np.asarray(data)
    all_features = np.vstack((all_features,data))

df = pandas.DataFrame(all_features, index=range(all_features.shape[0]),
                          columns=range(all_features.shape[1]))

all_fnames=[]
for i in range(len(all_features)):
    fname = 'images/bubble_' + str(int(all_features[i][1])) + '_' + str(int(all_features[i][0])) + '.png'
    #all_features[i].append(fname)
    all_fnames.append(fname)
    
## add a last column with filenames
df[10] = all_fnames  

## write to disk
df.to_csv(outfname)
# np.savetxt(outfname, all_features, delimiter=',', fmt='%lf')
# print ('done generating csv')